In [1]:
import sys
import os

# Add the path to the parent directory of `src` to the sys.path
sys.path.append(os.path.abspath('C:\\Away\\FMP\\New\\guiV2_1-main_2\\src'))

%load_ext autoreload
%autoreload 2

from backend.backend.simulation import SimulationConfig, Simulation
from backend.backend.vesicle import VesicleConfig, Vesicle
from backend.backend.exterior import ExteriorConfig, Exterior
from backend.backend.ion_channels import (IonChannelConfig, 
                                          IonChannel, 
                                          ASORChannelConfig, 
                                          ASORChannel, 
                                          CLCChannelConfig, 
                                          CLCChannel, 
                                          TPCChannelConfig, 
                                          TPCChannel, 
                                          NHEChannelConfig, 
                                          NHEChannel, 
                                          VATPaseChannelConfig, 
                                          VATPaseChannel,
                                          HLeakChannelConfig,
                                          HLeakChannel,
                                          KChannelConfig,
                                          KChannel
                                        )

from backend.backend.ion_species import IonSpecies

from trash.old_config import initialize_internal_concentrations

import numpy as np
from math import exp, log

In [2]:
r=1.3e-6 # radius of the vesicle in m
V0 = (4.0/3.0) * np.pi * (r**3) # Initial volume of the vesicle in m3
A0 = 4.0 * np.pi * (r**2) # initial surface area of the vesicle in m2
A_from_V_const=(36.0*np.pi)**(1/3)
c_spec = 0.01  # membrane capacitance per surface area F/(m**2)
U0 = 40*1e-3 # membrane potential in V 
C0 = A0 * c_spec # inititial membrane capacitance
Q0 = U0 * C0
buffer_capacity_t0 = 5.0 * 1e-4  # initial buffer capacity
RT = 2578.5871  # gas constant x temperature in Kelvin
F = 96485.0  # Faraday constant
Cl_idx = 0
Na_idx = 1
H_idx = 2
K_idx = 3
number_of_ions = 4


Cl_o_concentration = 20 * 1e-3  # external Cl concentraion in M
Na_o_concentration = 10 * 1e-3  # external Na concentraion in M
Na_i_concentration = 150 * 1e-3  # internal Na concentraion in M
K_i_concentration = 5 * 1e-3  # internal K concentraion in M
K_o_concentration = 140 * 1e-3  # external Cl concentraion in M
pH_o = 7.2  # external pH
pH_i = 7.4  # internal pH
hfree_o_concentration = 10 ** (-pH_o)  # concentration of free external protons in M
hfree_i_concentration = 10 ** (-pH_i)  # concentration of free internal protons in M
htotal_o_concentration = (
    hfree_o_concentration / buffer_capacity_t0
)  # concentration of total external protons in M
htotal_i_concentration = (
    hfree_i_concentration / buffer_capacity_t0
)  # concentration of total intenral protons in M
htotal_i_amount = (
    htotal_i_concentration * V0 * 1000
)  # amount of total internal protons in moles

def initialize_internal_concentrations(Cl_i_concentration=159 * 1e-3):
    # Cl_i_concentration= 159*1e-3 # internal Cl concentraion in M
    # Cl_i_concentration=1*1e-3 # absent internal Cl concentration in M
    # Cl_i_concentration= 9*1e-3 # Cl replacement condition (internal Cl concentration in M)

    Q0 = U0 * C0  # initial total charge
    X_amount = (Q0 / F) - (
        (
            Na_i_concentration
            + K_i_concentration
            + htotal_i_concentration
            - Cl_i_concentration
        )
        * V0
        * 1000
    )  # initial amount of unaccouted ions in moles
    X_concentration = X_amount / (
        V0 * 1000
    )  # initial concentration of unaccounted ions in moles

    internal_ions_amounts = [
        Cl_i_concentration * V0 * 1000,
        Na_i_concentration * V0 * 1000,
        htotal_i_concentration * V0 * 1000,
        K_i_concentration * V0 * 1000,
    ]  # vector of amounts of ions in moles
    external_ions_concentrations = [
        Cl_o_concentration,
        Na_o_concentration,
        htotal_o_concentration,
        K_o_concentration,
    ]  # vector of concentrations of external ions
    internal_ions_concentrations = [
        Cl_i_concentration,
        Na_i_concentration,
        htotal_i_concentration,
        K_i_concentration,
    ]  # vector of concentrations of internal ions

    Sum_initial_amounts = (
        internal_ions_amounts[Cl_idx]
        + internal_ions_amounts[Na_idx]
        + abs(X_amount)
        + internal_ions_amounts[K_idx]
    )  # sum of amounts of all ions

    return (
        X_amount,
        external_ions_concentrations,
        internal_ions_amounts,
        internal_ions_concentrations,
        Sum_initial_amounts,
    )

In [3]:
species_params = {
    'cl': {'exterior_conc': 20*1e-3, 'init_vesicle_conc': 159*1e-3, 'elementary_charge': -1},
    'na': {'exterior_conc': 10*1e-3, 'init_vesicle_conc': 150*1e-3, 'elementary_charge': 1},
    'k' : {'exterior_conc': 140*1e-3, 'init_vesicle_conc': 5*1e-3, 'elementary_charge': 1},
    'h' : {'exterior_conc': 12.619146889603859*1e-5, 'init_vesicle_conc': 7.962143411069938*1e-5, 'elementary_charge': 1}
}

cl_channels_params = {
    'ASOR': {
        'type': ASORChannel,
        'init_values': {
            'flux_multiplier': 1,
        },
    },
    'CLC': {
        'type': CLCChannel,
        'init_values': {
            'flux_multiplier': 2,
        },
    },      
}

na_channels_params = {
    'NHE': {
        'type': NHEChannel,
        'init_values': {
            'flux_multiplier': 1,
        },
    },
    'TPC': {
        'type': TPCChannel,
        'init_values': {
            'flux_multiplier': 1,
        },
    }, 
}

k_channels_params = {
    'K': {
        'type': KChannel,
        'init_values': {
            'flux_multiplier': 1,
        },
    },
}

h_channels_params = {
    'CLC': {
        'type': CLCChannel,
        'init_values': {
            'flux_multiplier': -1,
        },
    },
    'NHE': {
        'type': NHEChannel,
        'init_values': {
            'flux_multiplier': -1,
        },
    }, 
    'VATPase': {
        'type': VATPaseChannel,
        'init_values': {
            'flux_multiplier': -1,
        },
    },
    'HLeak': {
        'type': HLeakChannel,
        'init_values': {
            'flux_multiplier': 1,
        },
    },
}

all_channels_params = {
    'cl': cl_channels_params,
    'na': na_channels_params,
    'k': k_channels_params,
    'h': h_channels_params
}

Test_Vesicle = Vesicle(display_name='vesicle')
Test_Exterior = Exterior(display_name='exterior')

Sim = Simulation()

Sim.add_vesicle(Test_Vesicle)
Sim.add_exterior(Test_Exterior)

In [4]:
for ion_name, ion_params in species_params.items():
    ion_species = IonSpecies(display_name=ion_name, **ion_params)
    if ion_name == 'h':
        Sim.add_hydrogen_species(ion_species)
        
    Sim.add_ion_species(ion_species)
    
    for channel_name, channel_params in all_channels_params[ion_name].items():
        channel = channel_params['type'](display_name=f'{ion_name}_{channel_name}', **channel_params['init_values'])
        Sim.add_channel(ion_species, channel)

In [5]:
Sim.set_ion_amounts()
Sim.update_simulation_state()
Sim.update_vesicle_concentrations()

In [6]:
print(f'first step sim volume: {Sim.vesicle.volume}')
print(f'Masha init volume: {V0}')

print(f'first step sim area: {Sim.vesicle.area}')
print(f'Masha init area: {A0}')

print(f'first step sim capacitance: {Sim.vesicle.capacitance}')
print(f'Masha init capacitance: {C0}')

print(f'first step sim voltage: {Sim.vesicle.config.voltage}')
print(f'Masha init voltage: {U0}')

print(f'first step sim charge: {Sim.vesicle.charge}')
print(f'Masha init charge: {Q0}')

for ion in Sim.all_species:
    print(ion.vesicle_conc, ion.exterior_conc)

print(f'first step sim buffer: {Sim.buffer_capacity}')
print(f'Masha init buffer: {buffer_capacity_t0}')

print(f'first step sim pH: {Sim.vesicle.pH}')
print(f'Masha init pH: {pH_i}')

first step sim volume: 9.202772079915702e-18
Masha init volume: 9.202772079915702e-18
first step sim area: 2.1237166338267028e-11
Masha init area: 2.1237166338267002e-11
first step sim capacitance: 2.1237166338267028e-13
Masha init capacitance: 2.1237166338267003e-13
first step sim voltage: 0.039999999999974674
Masha init voltage: 0.04
first step sim charge: 8.494866535301432e-15
Masha init charge: 8.494866535306801e-15
0.159 0.02
0.15 0.01
0.005 0.14
7.962143411069939e-05 0.0001261914688960386
first step sim buffer: 0.0005
Masha init buffer: 0.0005
first step sim pH: 7.4
Masha init pH: 7.4


In [7]:
flux_calculation_parameters = Sim.get_Flux_Calculation_Parameters()
fluxes = [ion.compute_total_flux(flux_calculation_parameters=flux_calculation_parameters) for ion in Sim.all_species]

Sim.histories.update_histories()
Sim.histories.display_histories()

vesicle_pH: [7.4]
vesicle_volume: [9.202772079915702e-18]
vesicle_area: [2.1237166338267028e-11]
vesicle_capacitance: [2.1237166338267028e-13]
vesicle_charge: [8.494866535301432e-15]
exterior_pH: [7.2]
cl_vesicle_conc: [0.159]
cl_vesicle_amount: [1.4632407607065967e-15]
cl_ASOR_flux: [-1.07357053759734e-22]
cl_ASOR_nernst_potential: [-0.015406067174621264]
cl_ASOR_pH_dependence: [0.0024726231566347743]
cl_ASOR_voltage_dependence: [0.0016588010801777777]
cl_ASOR_time_dependence: [None]
cl_CLC_flux: [-6.925849492463746e-24]
cl_CLC_nernst_potential: [-0.0010398577539806053]
cl_CLC_pH_dependence: [0.9453186827840592]
cl_CLC_voltage_dependence: [0.0016588010801777777]
cl_CLC_time_dependence: [None]
na_vesicle_conc: [0.15]
na_vesicle_amount: [1.3804158119873552e-15]
na_NHE_flux: [-0.0]
na_NHE_nernst_potential: [-3.2085672197009947]
na_TPC_flux: [-4.7729833335987726e-18]
na_TPC_nernst_potential: [-0.11237335663276282]
k_vesicle_conc: [0.005]
k_vesicle_amount: [4.601386039957851e-17]
k_K_flux: